<a href="https://colab.research.google.com/github/O-suke12/Semantic_Segmentation/blob/main/Semantic_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from PIL import Image
import os
from google.colab import files

if torch.cuda.is_available(): 
 dev = "cuda:0" 
else: 
  dev = "cpu" 
dev

'cpu'

# **Reference**





Dataset: https://www.kaggle.com/datasets/bulentsiyah/semantic-drone-dataset?sort=votes

# **Dataset**

In [2]:
if os.path.exists("kaggle.json"):
  print("already exist")
else:
  uploaded = files.upload()
  ! mkdir ~/.kaggle
  ! cp kaggle.json ~/.kaggle/
  ! chmod 600 ~/.kaggle/kaggle.json
  ! kaggle datasets download -d bulentsiyah/semantic-drone-dataset
  ! unzip  semantic-drone-dataset

Saving kaggle.json to kaggle.json
100% 3.88G/3.89G [00:46<00:00, 89.8MB/s]
100% 3.89G/3.89G [00:46<00:00, 90.4MB/s]
Archive:  semantic-drone-dataset.zip
  inflating: RGB_color_image_masks/RGB_color_image_masks/000.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/001.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/002.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/003.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/004.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/005.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/006.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/008.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/011.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/013.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/014.png  
  inflating: RGB_color_image_masks/RGB_color_image_masks/015.png  
  inflating: RGB_color_image_masks/RGB_colo

In [3]:
df_class = pd.read_csv("/content/class_dict_seg.csv")
df_class.head()

,name,r,g,b
0,unlabeled,0,0,0
1,paved-area,128,64,128
2,dirt,130,76,0
3,grass,0,102,0
4,gravel,112,103,87


In [4]:
import os
from sklearn.model_selection import train_test_split

dict = {"Name":[]}
df = pd.DataFrame(dict)
df = pd.DataFrame(os.listdir("/content/RGB_color_image_masks/RGB_color_image_masks"))
df_train, df_valid = train_test_split(df, test_size=0.2, random_state=42)

In [37]:
from torch.utils.data import Dataset
from torchvision.io import read_image
from  torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize(size=(400,600)),
    transforms.ConvertImageDtype(torch.float),
])

class Custom_dataset(Dataset):
  def __init__(self, image_dir, rgb_dir,df ,transform=True):
    self.df = df
    self.transform = transform

  def __len__(self):
    return len(self.df)

  def __getitem__(self,idx):
    image = read_image(image_dir+"/"+self.df.iloc[idx,0][:-3]+"jpg")
    rgb = read_image(rgb_dir+"/"+self.df.iloc[idx,0])
    image = self.transform(image)
    rgb = self.transform(rgb)
    return image, rgb

image_dir = "/content/dataset/semantic_drone_dataset/original_images"
rgb_dir = "/content/RGB_color_image_masks/RGB_color_image_masks"

CD_train = Custom_dataset(image_dir, rgb_dir, df_train, preprocess)
CD_train.__getitem__(0)

(tensor([[[0.9020, 0.8980, 0.8980,  ..., 0.2039, 0.1961, 0.1412],
          [0.9020, 0.8980, 0.8980,  ..., 0.2118, 0.1804, 0.1725],
          [0.9020, 0.9059, 0.9020,  ..., 0.1922, 0.1961, 0.1804],
          ...,
          [0.3137, 0.4078, 0.0824,  ..., 0.1294, 0.2824, 0.3176],
          [0.2667, 0.1137, 0.2235,  ..., 0.3529, 0.3451, 0.3529],
          [0.2627, 0.1843, 0.2667,  ..., 0.2784, 0.3137, 0.3098]],
 
         [[0.9333, 0.9373, 0.9373,  ..., 0.2196, 0.2078, 0.1725],
          [0.9412, 0.9451, 0.9373,  ..., 0.2275, 0.2118, 0.1961],
          [0.9412, 0.9373, 0.9412,  ..., 0.2196, 0.2118, 0.2039],
          ...,
          [0.2078, 0.3569, 0.0745,  ..., 0.1098, 0.2784, 0.3098],
          [0.1804, 0.0863, 0.2745,  ..., 0.3569, 0.3686, 0.3647],
          [0.1490, 0.1216, 0.2275,  ..., 0.3098, 0.2980, 0.3137]],
 
         [[0.9412, 0.9412, 0.9412,  ..., 0.2667, 0.2745, 0.2118],
          [0.9451, 0.9451, 0.9412,  ..., 0.2706, 0.2667, 0.2510],
          [0.9451, 0.9451, 0.9412,  ...,

# **Model**

In [41]:
class double_conv(nn.Module):
  def __init__(self, in_features, out_features):
    super().__init__()
    self.conv_block = nn.Sequential(
        nn.Conv2d(in_features, out_features, 3),
        nn.BatchNorm2d(out_features),
        nn.ReLU(),
        nn.Conv2d(out_features, out_features, 3),
        nn.BatchNorm2d(out_features),
        nn.ReLU(),
    )
    

  def __forward(self,X):
    self.conv_block(X)
    return X

D1 = double_conv()

class UNET(nn.Module):
  def __init__():
    super().__init__()
    self.
 